In [ ]:
import sys
import serial
import time
import csv
from PyQt5 import uic
from PyQt5.QtWidgets import QMainWindow, QApplication, QMessageBox
from PyQt5.QtCore import QTimer

class ArduinoApp(QMainWindow):
    def __init__(self):
        super().__init__()
        
        # Cargar Interfaz (asegúrate de tener el archivo integracion.ui)
        try:
            uic.loadUi('integracion.ui', self)
        except:
            # Si no tienes la interfaz, crearemos controles básicos
            self.setupBasicUI()
        
        # Configurar el puerto serial
        try:
            self.arduino = serial.Serial('COM12', 9600, timeout=1)
            time.sleep(2)  # Esperar a que Arduino reinicie
            self.arduino.flush()
            self.connected = True
        except Exception as e:
            QMessageBox.critical(self, 'Error', f'No se pudo conectar al Arduino: {e}')
            self.connected = False
        
        # Configurar temporizador para leer datos
        self.timer = QTimer()
        self.timer.timeout.connect(self.read_data)
        self.timer.start(1000)  # Leer cada segundo
        
        # Conectar señales (ajusta según tu interfaz)
        try:
            self.slider_bulb.valueChanged.connect(self.send_power_bulb)
            self.slider_fan.valueChanged.connect(self.send_power_fan)
            self.btn_export.clicked.connect(self.csv_export)
        except:
            print("Algunos controles no están disponibles en la interfaz")
        
        # Almacenamiento de datos
        self.datos = []
        
    def setupBasicUI(self):
        # Crear una interfaz básica si no tienes el archivo .ui
        from PyQt5.QtWidgets import QWidget, QVBoxLayout, QHBoxLayout, QSlider, QLabel, QPushButton
        from PyQt5.QtCore import Qt
        
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout()
        
        # Control para bombilla
        bulb_layout = QHBoxLayout()
        bulb_layout.addWidget(QLabel("Bombilla (0-100%):"))
        self.slider_bulb = QSlider(Qt.Horizontal)
        self.slider_bulb.setRange(0, 100)
        bulb_layout.addWidget(self.slider_bulb)
        self.label_bulb = QLabel("0")
        bulb_layout.addWidget(self.label_bulb)
        layout.addLayout(bulb_layout)
        
        # Control para ventilador
        fan_layout = QHBoxLayout()
        fan_layout.addWidget(QLabel("Ventilador (0-100%):"))
        self.slider_fan = QSlider(Qt.Horizontal)
        self.slider_fan.setRange(0, 100)
        fan_layout.addWidget(self.slider_fan)
        self.label_fan = QLabel("0")
        fan_layout.addWidget(self.label_fan)
        layout.addLayout(fan_layout)
        
        # Botón para exportar
        self.btn_export = QPushButton("Exportar a CSV")
        layout.addWidget(self.btn_export)
        
        # Etiqueta para temperatura
        self.label_temp = QLabel("Temperatura: -- °C")
        layout.addWidget(self.label_temp)
        
        central_widget.setLayout(layout)
        self.setWindowTitle("Control de Incubadora")
        
    def read_data(self):
        if not self.connected:
            return
            
        try:
            while self.arduino.in_waiting > 0:
                line = self.arduino.readline().decode('utf-8', errors='ignore').rstrip()
                if line.startswith('T'):
                    try:
                        temperatura = float(line[1:])
                        self.label_temp.setText(f"Temperatura: {temperatura:.2f} °C")
                        # Guardar dato con timestamp
                        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
                        self.datos.append([temperatura, self.slider_bulb.value(), 
                                            self.slider_fan.value(), timestamp])
                    except ValueError:
                        print(f"Error al convertir temperatura: {line}")
        except Exception as e:
            print(f"Error leyendo datos: {e}")
            self.connected = False
            QMessageBox.critical(self, 'Error', f'Error de comunicación: {e}')

    def send_power_bulb(self, power):
        if self.connected:
            try:
                self.arduino.write(f'P{power}\n'.encode('utf-8'))
                self.label_bulb.setText(str(power))
            except Exception as e:
                print(f"Error enviando potencia a bombilla: {e}")
                self.connected = False

    def send_power_fan(self, power):
        if self.connected:
            try:
                self.arduino.write(f'F{power}\n'.encode('utf-8'))
                self.label_fan.setText(str(power))
            except Exception as e:
                print(f"Error enviando potencia a ventilador: {e}")
                self.connected = False

    def csv_export(self):
        try:
            with open('datos_incubadora.csv', 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Temperatura', 'Potencia Bombilla', 'Potencia Ventilador', 'Fecha'])
                for fila in self.datos:
                    writer.writerow(fila)
            QMessageBox.information(self, 'Éxito', 'Datos exportados a datos_incubadora.csv')
        except Exception as e:
            QMessageBox.critical(self, 'Error', f'No se pudo exportar los datos: {e}')

    def closeEvent(self, event):
        if self.connected and self.arduino.is_open:
            # Apagar todo al cerrar
            self.arduino.write('P0\n'.encode('utf-8'))
            self.arduino.write('F0\n'.encode('utf-8'))
            self.arduino.close()
        event.accept()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = ArduinoApp()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\vague\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
